In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from tensorflow_probability import distributions as tfd
import arviz as az

In [2]:
def prior_kernel(s,t):
    return np.exp(-(t-s)**2)

In [3]:
class posterior:
    def __init__(self, T, x, y, prior_kernel,sigma):
        self.T = T
        self.x = x
        self.y = y
        self.prior_kernel = prior_kernel
        self.n = y.shape[0]
        self.g = T.shape[0]
        self.sigma = sigma
        self.K = np.fromfunction(np.vectorize(lambda s, t: self.prior_kernel(self.T[s], self.T[t])), (self.g, self.g), dtype=int)
    
    def Lx(self,t):   
        L = np.zeros(self.n)    
        for i in np.arange(0,self.n):
            L[i] = np.trapz(self.K[t, :] * self.x[i,:], self.T)
        return L
        
    def R(self,i, j):
        Ri = np.trapz(self.K[:, :] * self.x[i, :], self.T, axis=1)
        Rij = np.trapz(Ri * self.x[j, :],self.T)
        return Rij

    def cov(self):
        return np.array([[self.R(i, j) for j in range(self.n)] for i in range(self.n)])
    
    # return m(t) and K*(s,t) for every s,t in T (TODO)
    def posterior_mean_cov(self):
        inv = np.linalg.inv(self.cov() + self.sigma * np.identity(self.n))
        def m(t):
            mean = np.dot(np.dot(np.transpose(self.Lx(t)),inv),self.y)
            return mean
        def Kstar(s,t):
            cov = self.K[s,t] - np.dot(np.dot(np.transpose(self.Lx(s)),inv),self.Lx(t))
            return cov
        
        m = np.fromfunction(np.vectorize(lambda t: m(t)), (self.g,), dtype=int)
        C = np.fromfunction(np.vectorize(lambda t, s: Kstar(s,t)), (self.g,self.g), dtype=int)
        return m,C
    
    def only_mean(self):
        inv = np.linalg.inv(self.cov() + self.sigma * np.identity(self.n))
        def m(t):
            mean = np.dot(np.dot(np.transpose(self.Lx(t)),inv),self.y)
            return mean
        m = np.fromfunction(np.vectorize(lambda t: m(t)), (self.g,), dtype=int)
        return m
    
    def fitted_values(self):
        m = self.only_mean()
        return np.fromfunction(np.vectorize(lambda i: np.trapz(self.x[i,:]*m,self.T)), (self.n,), dtype=int)

In [4]:
df = pd.read_csv("./data/average_curves.csv", index_col='idx')
df

,0,1,2,3,4,5,6,7,8,9,...,150,151,152,153,154,155,156,157,158,159
idx,,,,,,,,,,,,,,,,,,,,,
pat_number,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,...,150.000000,151.000000,152.000000,153.000000,154.000000,155.000000,156.000000,157.000000,158.000000,159.000000
severity,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
constancy,1.000000,1.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
age,58.000000,59.000000,62.000000,65.000000,65.000000,70.000000,66.000000,74.000000,58.000000,74.000000,...,68.000000,49.000000,51.000000,66.000000,45.000000,74.000000,67.000000,59.000000,34.000000,72.000000
gender,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,-0.981659,-1.076615,-1.332550,-1.058805,-0.923807,-1.109480,-1.743754,-1.184359,-1.359995,-1.107242,...,-1.485589,-1.074087,-1.003103,-0.996204,-1.740179,-1.447827,-1.043098,-1.684661,-1.077221,-1.393487
284,-0.993531,-1.078280,-1.332549,-1.044116,-0.923762,-1.109441,-1.743714,-1.217882,-1.360021,-1.116414,...,-1.486356,-1.074077,-1.002612,-0.994606,-1.740226,-1.445859,-1.042748,-1.684662,-1.077203,-1.393533
285,-0.997926,-1.079618,-1.332558,-1.019142,-0.923818,-1.109399,-1.743523,-1.224489,-1.360155,-1.127783,...,-1.487368,-1.074091,-1.003404,-0.994287,-1.740254,-1.440550,-1.042608,-1.684662,-1.077191,-1.393537


In [5]:
x = df.iloc[7:, :].T.to_numpy()
y = df.iloc[1, :].T.to_numpy()

In [6]:
y23 = np.copy(y)
y23[y == 1] = 0
y23[y == 2] = 1
y23[y == 3] = 1
np.unique(y23)

array([0., 1.])

The __log pseudo-marginal likelihood__ is given by
$$
    LPML_j = \sum_{i=1}^n log(CPO_i|M_j)
$$
where
$$
 CPO_i | M_j = m(y_i | \textbf{y}_{-i}, M_j)
$$
Calculation is done using (Guglielmi's notes pag. 48)
$$
 (CPO_i)^{-1} = \int_{\Theta} \frac{1}{f(y_i | \theta)} \pi(\theta|y) d \theta
$$
and approximating $CPO_i$ using MC as
$$
    (CPO_i)^{-1} \thickapprox \frac{1}{M}\sum_{t=1}^M \frac{1}{f(y_i | \theta^{(t)})} 
$$
where 
$$
     \theta^{(t)} \sim \pi(\theta|y)
$$
In our case $\theta = \beta$ and for the __unconstrained model__ we have assumed the following:
$$
    f(y_i | \beta) = N(\int x_i(t)\beta(t)dt, \sigma^2)
$$
$$
    \pi(\beta | y) = GP(m, K^*)
$$
$$
    m(t) = Lx(t)^{'}(\Sigma + \sigma^2I_n)^{-1}Y
$$
$$
    K^*(s,t) = K(s,t) - Lx(s)^{'}(\Sigma + \sigma^2I_n)^{-1}Lx(t)
$$
Doubts are:
- Does the formulation we used for $f(y_i|\beta)$ is the one appropriate for CPO calculation? At the end of the fitting, we use a logit to produce the "final" fitted values. Does this influence the likelihood we have to use for CPO and in which way?

- we interpreted the terms $f(y_i | \theta^{(t)})$ in the CPO calculation as evaluating in the point $y_i$ the likelihood under the parameter sampled from the posterior. Does this $y_i$ are the real target or instead the fitted target?



In [7]:
sigma = 1
T = np.linspace(1, 288, 288)
model = posterior(T, x, y23, prior_kernel,sigma)

In [9]:
m, Kst = model.posterior_mean_cov();

In [10]:
M = 50
sigma = 1
LPML = 0

for i in np.arange(50):
    sum = 0
    
    for t in np.arange(M):
        beta_post = np.random.multivariate_normal(m,Kst)
        mean = np.trapz(x[i, :]*beta_post,T)
        f = tfd.Normal(mean, sigma)
        f_i = f.prob(y23[i])
        sum += 1 / f_i 

    CPO_i_inv = sum / M
    CPO_i = 1 / CPO_i_inv
    
    log_CPO_i = np.log(CPO_i)
    LPML += log_CPO_i

2024-01-30 17:01:24.048122: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
LPML

-68.15489102340325

In [12]:
fitted = model.fitted_values()

In [15]:
M = 50
sigma = 1
LPML1 = 0

for i in np.arange(50):
    sum = 0
    
    for t in np.arange(M):
        beta_post = np.random.multivariate_normal(m,Kst)
        mean = np.trapz(x[i, :]*beta_post,T)
        f = tfd.Normal(mean, sigma)
        f_i = f.prob(scipy.special.expit(fitted[i]))
        sum += 1 / f_i 

    CPO_i_inv = sum / M
    CPO_i = 1 / CPO_i_inv
    
    log_CPO_i = np.log(CPO_i)
    LPML1 += log_CPO_i

In [16]:
LPML1

-65.80554147074572